In [258]:
from collections import defaultdict
from random import randint
from copy import copy

class Worker:
    def __init__(self, id):
        self.worker_id = id

available_games = 10
workers = []
for id in range(5):
    workers.append(Worker(id))
    
worker_record = defaultdict(lambda: [])
worker_blocks = defaultdict(lambda: [])
worker_bans = []

worker_names = ["Avery", "Jordan", "Blake", "River", "Eden", "Phoenix", "Harley", "Alexis", "Parker", "Taylor"]

VERBOSE = True
MAX_GAMES = 10

game_id = None

In [290]:
def set_game_id(id):
    """
    Sets the global variable game_id to a given value
    :param id: a new value for global variable game_id
    :return: Nothing.
    """
    global game_id
    # print("Setting game ID to {}".format(game_id))
    game_id = id

def select_random_game(exceptions = None):
    """
    Returns a random game ID
    :return: a random game ID
    """
    global available_games
    
    while True:
        game_id = randint(0, available_games)
#         print("Game ID is {}. Exceptions are {}".format(game_id, exceptions))
        if exceptions != None:
            exceptions = set(exceptions)
            if game_id not in exceptions:
                break
        else: break
#     print("Selected random game {}".format(game_id))
    return game_id

def update_records(players, game_id):
    set_game_id(game_id)
    
    # Add game ID to the worker record and to the blocked list if it is the second occurance.
    # If a player has played 10 games, he or she gets banned from the game.
    worker_record[players[0].worker_id].append(game_id)
    if worker_record[players[0].worker_id].count(game_id) == 2:
         worker_blocks[players[0].worker_id].append(game_id)
    if len(worker_record[players[0].worker_id]) == 10:
        worker_bans.append(players[0].worker_id)

    worker_record[players[1].worker_id].append(game_id)
    if worker_record[players[1].worker_id].count(game_id) == 2:
         worker_blocks[players[1].worker_id].append(game_id)
    if len(worker_record[players[1].worker_id]) == 10:
        worker_bans.append(players[1].worker_id)

def check_workers_eligibility(workers):
    """
    Checks the list of available workers, pairs the first two players that pass
    the game criteria and sets the corresponding game id
    :param workers: a list of all available workers
    :return: a list of two workers that are paired for the next game
    """
    
    global VERBOSE
    global MAX_GAMES

    global worker_record
    global worker_blocks
    global worker_bans

    game_id = None

    players = []

    # Return an empty list if not enough workers are in queue
    if len(workers) < 2:
        return []
    
    if VERBOSE: print("{} workers available:".format(len(workers)))
    idx = 0
    worker = workers[idx]
    worker_id = worker.worker_id
    players.append(worker)
    if VERBOSE: print("Worker: {}".format(worker_id))

    # Worker never played before. Assign the next queued worker as partner
    if worker_id not in worker_record.keys():
        if VERBOSE: print("Worker has no recorded games")

        partner = workers[idx+1]
        partner_id = partner.worker_id
        players.append(partner)
        if VERBOSE: print("Partner: {}".format(partner_id))

        # Partner also has no games, so start a random one
        if partner_id not in worker_record.keys():
            game_id = select_random_game()
            if VERBOSE: print("Partner has no recorded games. Setting game ID randomly to {}".format(game_id))
        # Partner has played before
        else:
            last_game = worker_record[partner_id][-1]
            # Check if his or her last game is not yet blocked and select it
            if last_game not in worker_blocks[partner_id]: game_id = played_id
            # Otherwise select a random game with exception of those played by the partner
            else: game_id = select_random_game(exceptions=worker_record[partner_id])
            if VERBOSE: print("Partner has recorded games. Setting game ID to {}".format(game_id))

        update_records(players, game_id)
        return players

    # If worker played before, make sure that the partner has never played the same game
    else:
        last_game = worker_record[worker_id][-1]
        # Check if the worker's last game is not yet blocked
        if last_game not in worker_blocks[worker_id]:
            game_id = last_game
            max_index = max(idx+3, len(workers)-2)
            # Check the next three players in the queue whether they can play the worker's last game
            for partner in workers[idx+1:max_index]:                          
                partner_id = partner.worker_id
                if VERBOSE: print("Checking potential partner {}".format(partner_id))
                # If partner never played the worker's last game, pair them. Otherwise continue looping
                if game_id not in worker_record[partner_id]:
                    players.append(partner)
                    if VERBOSE: print("Partner has not played this game before.")
                    update_records(players, game_id)
                    return players

            # If no partner could be found for the worker's last game, select the next worker in the queue and
            # select a random game that none of the two played before
            partner = workers[idx+1]
            partner_id = partner.worker_id
            players.append(partner)
            if VERBOSE: print("Partner: {}".format(partner_id))
            blocked = copy(worker_record[worker_id])
            blocked.extend(worker_record[partner_id])
            game_id = select_random_game(exceptions=blocked)
            if VERBOSE: print("Selected game {} as it was not played by any of the players before".format(game_id))
            update_records(players, game_id)
            return players

        # Else try the last game of the next player in the queue
        else:
            partner = workers[idx+1]
            partner_id = partner.worker_id
            players.append(partner)
            if VERBOSE: print("Partner: {}".format(partner_id))
            last_game = worker_record[partner_id][-1]
            # Check if the partner's last game is not yet blocked and never played by the worker
            if last_game not in worker_blocks[partner_id] and last_game not in worker_record[worker_id]:
                game_id = played_id
                if VERBOSE: print("Partner has recorded games. Setting game ID to {}".format(game_id))
            # Else select a random one that none of the two played before    
            else:
                blocked = copy(worker_record[worker_id])
                blocked.extend(worker_record[partner_id])
                game_id = select_random_game(exceptions=blocked)
                if VERBOSE: print("Selected game {} as it was not played by any of the players before".format(game_id))
            
            update_records(players, game_id)
            return players
        
    if VERBOSE: print("Houston, we have a problem. All is lost.")
    return [] 

def check_bans():
    global worker_record
    global worker_bans
    
    for worker_id, played_games in worker_record.items():
        if len(played_games) == MAX_GAMES:
            worker_bans.append(worker_id)
            
def filter_workers(workers):
    global worker_bans
    
    filtered_workers = []
    for worker in workers:
        if worker.worker_id not in worker_bans:
            filtered_workers.append(worker)
    return filtered_workers   

def get_worker_names(players):
    global worker_names
    
    player_names = []
    for player in players:
        player_id = player.worker_id
        n = len(worker_record[player_id])-1
        print(n)
        player_names.append(worker_names[n])
        
    return player_names
    

In [349]:
print('Records:')
for key, value in worker_record.items():
    print("{}: {}".format(key, value))

print('Blocks:')
for key, value in worker_blocks.items():
    print("{}: {}".format(key, value))
    
print('Bans:')
print(list(set(worker_bans)))

Records:
0: [0, 0, 9, 9, 8, 8, 1, 1, 5, 5]
1: [0, 9, 8, 1, 5, 5, 4, 4, 3, 3]
2: [0, 9, 8, 1, 5, 4, 3, 3, 7, 7]
3: [5, 4, 3, 7, 0, 6, 8, 2, 9, 1]
4: [3, 7, 0, 6, 8, 2, 9, 1]
Blocks:
0: [0, 9, 8, 1, 5]
1: [5, 4, 3]
2: [3, 7]
3: []
Bans:
[0, 1, 2, 3]


In [350]:
workers = filter_workers(workers)
players = check_workers_eligibility(workers)
check_bans()
player_names = get_worker_names(players)
print(player_names)

[]


## Reset statistics

In [294]:
workers = []
for id in range(5):
    workers.append(Worker(id))
    
worker_record = defaultdict(lambda: [])
worker_blocks = defaultdict(lambda: [])
worker_bans = []